# Model Creation

## Import libraries

In [3]:
pip install torch


  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB 100.9 kB/s eta 0:26:24
   ---------------------------------------- 0.0/159.8 MB

In [4]:
import matplotlib.pyplot as plt
import requests
import urllib.request as urllib
from PIL import Image
from io import BytesIO

import torch
from torchvision.io import read_image
from torchvision import transforms, models
import torch.nn.functional as F
import torch.nn as nn

ModuleNotFoundError: No module named 'torchvision'

## Create: Classification network

In [ ]:
weights = models.MobileNet_V3_Small_Weights.DEFAULT
classification_model = models.mobilenet_v3_small(weights=weights)

In [ ]:
preprocess = weights.transforms()
classification_model.eval()

## Test: Classification Network

In [ ]:
def process_img_from_path(img_path):
  img = read_image(img_path)
  x = preprocess(img).unsqueeze(0)
  return x

def process_img_from_array(img_array):
  x = preprocess(img_array).unsqueeze(0)
  return x

def process_img_from_PIL(img):
  x = transforms.ToTensor()(img)
  x = preprocess(x).unsqueeze(0)
  return x

In [ ]:
x = process_img_from_path("test_imgs/cat.jpg")

In [ ]:
# Predict the class probabilities
prediction = classification_model(x).squeeze(0).softmax(0)

class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

## Create: Siamese network

In [ ]:
base_net = models.mobilenet_v3_small(weights=weights)

# Remove the fully connected layer (classification head)
base_net = nn.Sequential(*list(base_net.children())[:-1])

# Set the model to evaluation mode
base_net.eval()

# Forward pass through the model
embeddings = base_net(x)

# Print the output tensor
print(embeddings.shape)

In [192]:
class SiameseNetwork(nn.Module):
  def __init__(self):
    super(SiameseNetwork, self).__init__()

    self.net = base_net

  def forward_once(self, x):
    return self.net(x)

  def forward(self, input1, input2):
    # calculate embeddings
    output1 = self.forward_once(input1)
    output2 = self.forward_once(input2)

    # calculate euclidean distance
    dist = torch.sum(F.pairwise_distance(output1, output2))
    return dist.item()

siamese_model = SiameseNetwork()

## Test: Siamese Network

In [ ]:
siamese_model.eval()

In [ ]:
dog = process_img_from_path("test_imgs/dog.jpg")
dog2 = process_img_from_path("test_imgs/dog2.jpg")
cat = process_img_from_path("test_imgs/cat.jpg")
cat2 = process_img_from_path("test_imgs/cat2.jpg")

dog.shape, dog2.shape, cat.shape, cat2.shape

Same class comparisons:

In [ ]:
siamese_model(dog, dog2)

In [196]:
siamese_model(cat, cat2)

203.96353149414062

Different class comparisons:

In [197]:
siamese_model(dog, cat)

329.5746154785156

In [198]:
siamese_model(dog, cat2)

361.8633117675781

In [199]:
siamese_model(dog2, cat)

338.45770263671875

In [200]:
siamese_model(dog2, cat2)

364.199462890625

It is clear that a lower similarity score is attained when images of the same class are compared and a higher similarity score is attained when images of different classes are compared.

The lower the similarity score, the more similar the images are.

# Setup Data Fetching

## Setup: Unsplash API

In [ ]:
client_id = "1Rb3JZ4ZLqv1ps70yBqjxmpvWkoOeORD2mnh5UekJGk"

In [ ]:
def fetch_imgs(keyword, n_pages=3):
  img_urls = []
  for i in range(1, n_pages+1):
    url = f"https://api.unsplash.com/search/photos?page={i}&query={keyword}&client_id={client_id}"
    results = requests.get(url).json()["results"]
    for x in results:
      img_urls.append(x["urls"]["small"])
  return img_urls

In [ ]:
def load_img_from_url(url):
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  return image

## Test: Unsplash API

In [ ]:
img_urls = fetch_imgs("labrador", 1)
img_urls

In [ ]:
img = load_img_from_url(img_urls[0])
plt.imshow(img)
plt.show()

# Simulate API request

Given an input image, we will do the following steps:
1. Classify it through the classification model
2. Retrieve 50 images of this classification using the Unsplash API
3. Use the Siamese network to determine the top 20 similar images of these 50 images
4. Return these 20 images

## Load an input image

In [19]:
x = load_img_from_url("https://www.thesprucepets.com/thmb/o76tYUlS1kJCb8H9VCyL4249ayo=/2036x1473/filters:fill(auto,1)/GettyImages-584178259-5a721010119fa80037ed3cce.jpg")
plt.imshow(x)
plt.show()

NameError: name 'load_img_from_url' is not defined

## Classification

In [20]:
x = process_img_from_PIL(x)

NameError: name 'process_img_from_PIL' is not defined

In [21]:
# Predict the class probabilities
prediction = classification_model(x).squeeze(0).softmax(0)

class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

NameError: name 'classification_model' is not defined

## Retrieve comparison images

In [22]:
img_urls = fetch_imgs(category_name, n_pages=5)
print(f"{len(img_urls)} images retrieved.")

NameError: name 'fetch_imgs' is not defined

## Compare images using siamese network

In [23]:
sims = []

for index, img_url in enumerate(img_urls):
  img = load_img_from_url(img_url)
  img = process_img_from_array(img)
  sim_score = siamese_model(x, img)
  sims.append({
      "comp": img,
      "url": img_url,
      "sim_score": sim_score,
  })
  print(f"Image {index} done.")

NameError: name 'img_urls' is not defined

## Find top 20 similar images

In [17]:
# Sort images by similarity score

sims.sort(key=lambda x: x["sim_score"])

In [18]:
# Get top 20 similar images

sim_images = sims[:20]
sim_images = [x["url"] for x in sim_images]

In [213]:
sim_images

['https://images.unsplash.com/photo-1549295264-617dec805276?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w0NjAxMzZ8MHwxfHNlYXJjaHwxNXx8dGFiYnl8ZW58MHx8fHwxNjg2OTg2NDA0fDA&ixlib=rb-4.0.3&q=80&w=400',
 'https://images.unsplash.com/photo-1616044543567-a31e09b178ee?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w0NjAxMzZ8MHwxfHNlYXJjaHwxfHx0YWJieXxlbnwwfHx8fDE2ODY5ODY0MDR8MA&ixlib=rb-4.0.3&q=80&w=400',
 'https://images.unsplash.com/photo-1474743437114-94141802864e?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w0NjAxMzZ8MHwxfHNlYXJjaHw0OXx8dGFiYnl8ZW58MHx8fHwxNjg2OTg2NDA2fDA&ixlib=rb-4.0.3&q=80&w=400',
 'https://images.unsplash.com/photo-1616589050164-7dbe2ed3fe17?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w0NjAxMzZ8MHwxfHNlYXJjaHw4fHx0YWJieXxlbnwwfHx8fDE2ODY5ODY0MDR8MA&ixlib=rb-4.0.3&q=80&w=400',
 'https://images.unsplash.com/photo-1648170723309-46a266549e73?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w0NjAxMzZ8MHwxfHNlYXJjaHw0N3x8dGFiYnl8ZW58MHx8fHwxNjg2OTg2NDA2fDA&ixlib=rb-4.0.

This list of image urls is what the API will return.